In [1]:
import glob
import json

In [2]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_dataset

import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset

2024-06-08 14:50:54.075941: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 14:50:55.079996: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
mp3_data = glob.glob('/home/letfd/lct2024_data/data/audios/*.mp3')
print(len(mp3_data))

106656


In [4]:
## get parsed audios:

trans_arr = list()
transcribed = glob.glob('/home/letfd/lct2024_data/data/transcript/*.json')
for path in transcribed:
    with open(path, 'r') as f:
        trans_file = json.load(f)
        trans_arr.extend(trans_file)


In [7]:
trans_arr[0]

{'0b75b0e74c308a4f840ceb349b23.mp3': {'text': ' Субтитры сделал DimaTorzok',
  'chunks': [{'timestamp': [0.0, 12.88],
    'text': ' Субтитры сделал DimaTorzok'}]}}

In [12]:
# with open('/home/letfd/lct2024_data/data/transcript/merged_transcript_3.json', 'w') as f:
#     json.dump(trans_arr, f)

In [13]:
# !cp '/home/letfd/lct2024_data/data/transcript/merged_transcript_3.json' .

In [14]:
### 
new_format_output = dict()
for row in trans_arr:
    new_format_output.update(row.items())

In [15]:
len(new_format_output)

106400

In [16]:
list(new_format_output.keys())[:10]

['0b75b0e74c308a4f840ceb349b23.mp3',
 '60630ce8485586a4e0e333dacec3.mp3',
 '9882ccef46d7821bd0fdced4d97b.mp3',
 '7576fd1644c3a159ed61b2af19ee.mp3',
 '3d973bba4dd1ac04e5867a2c37be.mp3',
 '75d9529f4d04ad83375047d99235.mp3',
 'e1574c3a4ca2a782f6a2708cd971.mp3',
 '617d38164623be8c6b6e3a4c7c5b.mp3',
 'fbda3d234e328022a3d09afbf169.mp3',
 '6dc405c7438199975b7433b83402.mp3']

In [17]:
# with open('whisper_results_100k.json', 'w') as f:
#     json.dump(new_format_output, f)

In [10]:
parsed_mp3s = list()
for t in trans_arr:
    parsed_mp3s.extend(t.keys())
parsed_mp3s = set(parsed_mp3s)
len(parsed_mp3s)

37452

In [11]:
list(parsed_mp3s)[:10]

['3d648ef746b0b64694b5d26cea9c.mp3',
 '5201069e439e81d740c2a8343c6a.mp3',
 'cd47d8ae44e5ae11c96eb0e9b294.mp3',
 'a737b697413cb87c54726453a37c.mp3',
 'd0a9f17d4e539b76cc3d07957d01.mp3',
 '3c7ab9b7441c8ee4ec00865d76dd.mp3',
 'c8a8c8ba4f7ba372771ba36fd521.mp3',
 '01f95de7464c9c1e5f771b10dc8b.mp3',
 '10eff3164f7394167130e29cb9b4.mp3',
 'ced057654d39993176b1deddfd93.mp3']

In [12]:
%%time

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


CPU times: user 4.68 s, sys: 3.67 s, total: 8.35 s
Wall time: 9min 57s


In [13]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=32,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [14]:
# 41.5 сек на 65 видео (small, bs=16)
# 1min 9s на 65 видео (small, bs=4)
# 33.1 сек на 65 видел (tiny)
# 1min 13s на 65 видео (large-v3, bs=16)
# 4min 2s на 128 видео (large-v3, bs=16)
# 1min 37s на 128 видео (large-v3, bs=32)


In [15]:
def gen_batch(arr, batch_size=1024):
    for i in range(0, len(arr), batch_size):
        yield(arr[i:i+batch_size])

In [16]:
import pathlib

In [17]:
mp3_data = [p for p in mp3_data if pathlib.Path(p).name not in parsed_mp3s]

In [18]:
len(mp3_data)

69204

In [18]:
import json
import time

t0 = time.time()
for idx, part in enumerate(gen_batch(mp3_data, batch_size=64)):
    ## pseudo wisper:
    # trans = [{'text': 'dummy', 'chunks':[]} for _ in part]
    
    try:
        trans = pipe(part, generate_kwargs={"language": "russian"})
        zipped = [{fp.split('/')[-1]: transcript} for fp, transcript in zip(part, trans)]
        with open(f"/home/letfd/lct2024_data/data/transcript/trans_{idx}.json", "w") as f:
            json.dump(zipped, f)
    except Exception as e:
        print(f"failed part {idx} with exception {e}")

    print(f'finished part {idx}; elapsed time: {((time.time() - t0) / 60):.3f} minutes')